# Policy-Gradients with the REINFORCE algorithm

**Background**:
We will train an agent using the REINFORCE policy-gradient algorithm to learn to balance a pole in the OpenAI gym [Cartpole environment](https://gym.openai.com/envs/CartPole-v0).

**Learning objectives**:
* Understand the policy-gradient approach to directly training a parameterised policy to maximise expected future rewards.
* Understand how the policy-gradient theorem allows us to derive a stochastic gradient estimator on the policy, defined in terms of samples drawn from the very same policy.

**What is expected of you**:
 * Go through the explanation, keeping the above learning objectives in mind.
 * Fill in the missing code ("#IMPLEMENT-ME") and train a model to solve the Cartpole-v0 environment in OpenAI gym (you solve it when reward=200).


# A Simple Policy-Gradient Cartpole Example

## Introduction

There are many different approaches to training RL agents. All of them summarise the environment based on its current **state**, and try to learn how to move from the current state to the next state in order to maximise the **rewards** that the agent will receive over time.Policy-based methods directly tries to learn what action to take next in order to maximise future rewards. 

Policy-based methods try to learn a policy $\pi_\theta(a | s)$ which outputs a distribution over the possible actions $a$, given the current state $s$ of the environment. The goal is to maximise all expected future rewards, under the learned policy:

\begin{align}
    J(\theta) &= \sum_{t=0}^{T}    \left[ \pi_\theta(a_t|s_t) R(s_t, a_t) \right] \\
                        &= \mathbb{E}_{a_t \sim \pi_\theta} \left[ R(s_t, a_t) \right].
\end{align}

Policy-**gradient** methods learn $\pi_\theta$ by directly differentiating this objective function in terms of $\theta$:

\begin{align}
    \nabla_\theta J(\theta) = \sum_{t=0}^{T}    \left[ \nabla_\theta \pi_\theta(a_t|s_t) R(s_t, a_t) \right].
    \end{align}

We want to solve this by sampling actions from our policy and trying these out in the environment. However the current formulation doesn't allow that. But rearranging the identity $\nabla_\theta \log \pi_\theta = \frac{\nabla_\theta \pi_\theta}{\pi_\theta}$ gives us the trick $\nabla_\theta \pi_\theta = \pi_\theta \nabla_\theta \log \pi_\theta$ which we can substitute into the above to get the **[policy-gradient theorem](http://www.scholarpedia.org/article/Policy_gradient_methods)** (also called **[REINFORCE](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)**):

\begin{align}
 \nabla_\theta J(\theta) &= \sum_{t=0}^{T}    \left[ \pi_\theta(a_t|s_t) \nabla_\theta \log \pi_\theta(a_t|s_t) R(s_t, a_t) \right] \\
 &= \mathbb{E}_{a_t \sim \pi(a|s_t)} \left[ \nabla_\theta \log \pi_\theta(a_t|s_t) R(s_t, a_t) \right] && \triangleright \textrm{Definition of expections.}
\end{align}


**NOTE**: 
* We have a policy $\pi_\theta(a|s)$ which tells the agent which action $a$ to take, given the state $s$, and it is parameterised in terms of parameters $\theta$.
* Our goal $J(\theta)$ is to maximise the rewards the agent receives over time by **choosing actions from this policy** that lead to high future rewards.
* We'll use gradient-based optimisation for solving for $\theta$. We therefore want the gradient of our objective wrt the policy parameters (**given actions drawn from our policy**): $\nabla_\theta J(\theta)$.
* We use a simple trick to rearrange this expression for the gradient into an expectation over actions drawn from our policy as it's being learned.
* Since we can now sample actions to take from $a \sim \pi_\theta(a | s)$, we can approximate this gradient using **[Monte-Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_integration)** methods:
    \begin{align}
        \nabla_\theta J(\theta) &= \mathbb{E}_{a_t \sim \pi(a|s_t)} \left[ \nabla_\theta \log \pi_\theta(a_t|s_t) R(s_t, a_t) \right] \\ 
        &\approx \frac{1}{k} \sum_k \nabla_\theta \log \pi_\theta(a_k|s_t) R(s_t, a_k)
    \end{align}
    
* If our agent parameterises its actions-distribution as a softmax, then we already know how to do $\nabla_\theta \log \pi_\theta(a_t|s_t)$. It's simply the negative log-likelihood or cross-entropy loss!

This algorithm is called **Monte-Carlo REINFORCE**, and is one type of policy-gradient algorithm. Let's use this to solve the Cartpole environment!


![REINFORCE](https://github.com/s-mawjee/aims-rl/blob/master/images/reinforce.png?raw=true "REINFORCE Pseudocode")


In [ ]:
!pip install pip-packages/gym-0.10.9.tar.gz
!pip install matplotlib

In [ ]:
from __future__ import absolute_import, division, print_function
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical 

# for auto-reloading external modules
# (if you're curious, see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

### PyTorch - Use CUDA GPU if available

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Helper Functions

In [ ]:
def reset_matplotlib():
    %matplotlib inline
    plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
    plt.rcParams['image.interpolation'] = 'nearest'
    plt.rcParams['image.cmap'] = 'gray'

reset_matplotlib()

## Define the Environment

In [ ]:
env = gym.make('CartPole-v0')
env.seed(0)

# IMPLEMENT-ME
# Print out observation space for gym environment
# Print out action space for gym environment

## The Policy-based Agent

We begin by parameterising the policy $\pi_\theta(a | s)$ as a simple neural network which takes the state (a vector of 4 elements provided by `gym`) as input, and produces a softmax over the possible actions as output. Simple enough. Refer to [torch.nn](https://pytorch.org/docs/stable/nn.html)

### Define the Architecture of the Policy

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        # IMPLEMENT-ME
        # Define neural network layers. Refer to nn.Linear (https://pytorch.org/docs/stable/nn.html#torch.nn.Linear)
        # Two Linear layers

    def forward(self, x):
        # IMPLEMENT-ME
        # Define neural network forward pass, 
        # with RELU function after first later 
        # and use softmax on last layer. Refer to torch.nn.functional.softmax (https://pytorch.org/docs/0.3.1/nn.html#torch.nn.functional.softmax)
        return pass
    
    def act(self, state):
        # Use policy model to pick an action
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

### Train the Agent with REINFORCE

Next we will write the code to update the model parameters given a reward. For this, we we need to sample an action from the model (policy), try it out in the environment, receive the reward from the environment, and use these together to get an estimate of the gradient wrt the policy. We then update the policy to do better the next time using [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam).

In [ ]:
learning_rate = 1e-2
number_episodes=1000
gamma = 1.0
max_episode_length = 1000

policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

def reinforce(n_episodes=number_episodes, max_episode_length=max_episode_length, gamma=gamma, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        
        # IMPLEMENT-ME
        # Generate an episodes 
        for t in range(max_episode_length):
            # Pick action and save log_prob using the policy
            # Act on env using the action to get the next_state and reward 
            rewards.append(reward)
            saved_log_probs.append(log_prob)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # IMPLEMENT-ME
        # Compute G 
        
        # IMPLEMENT-ME
        policy_loss = []
        for log_prob in saved_log_probs:
            # Compute `policy_loss` for each step, using G and log_prob 
            policy_loss = policy_loss * -1
        # Sum loss for episode
        policy_loss = torch.cat(policy_loss).sum()
        
        # Backprop using PyTorch
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores

In [ ]:
scores = reinforce()

### Plot scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### Watch Trained Agent

In [ ]:
state = env.reset()
for t in range(2000):
    action, _ = policy.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

In [ ]:
env.close()

# Reference
* Deep Learning Indaba: https://github.com/deep-learning-indaba
* Medium Blog: https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724
